### DESCRIPTION

Run S2A systems technoeconomic analysis. Run "s2a_systems_functions.py" for user-specified input parameters.

Outputs:
- "input params.csv": all sets of input parameters mapped to run #.
- "input params_varied.csv": input parameters that are different from baseline input parameters (run 0) mapped to run #.
- "output_[xxxx].csv": input parameters and results for each run. Filename suffix = run #.
- "output_all runs.csv": input parameters and results for all runs.
- "output_all runs_refueling station costs.csv": refueling station inputs and costs for "LOHC - formic acid" pathway for all runs.
- "output_all runs_refueling station costs_pivot.csv": refueling station inputs and costs for "LOHC - formic acid" pathway for all runs, pivot format.
- "output_cost by [pathway/process/location/function/process-function].csv": input parameters and cost results for all runs aggregated by pathway, process (e.g., preconditioning), location (e.g., terminal), function (e.g., compression), or by process and function.
- - "output_emissions by [pathway/process/location/function/process-function].csv": input parameters and emissions results for all runs aggregated by pathway, process (e.g., preconditioning), location (e.g., terminal), function (e.g., compression), or by process and function.

### IMPORT MODULES

In [1]:
import s2a_systems_functions as s2a_sys
import numpy as np
import pandas as pd
import os
from datetime import date

In [2]:
# show default arguments for reference
help(s2a_sys.calcs)

Help on function calcs in module s2a_systems_functions:

calcs(dict_input_params={'run #': 0, 'output dollar year': 2022, 'target station capacity (kg/day)': 1000.0, 'target number of stations': 10, 'one-way delivery distance (mile)': 100.0, 'electricity cost ($/kWh)': 0.1709, 'diesel cost ($/gallon)': 6.028, 'electricity emission factor (kg CO2/kWh)': 0.228, 'diesel emission factor (kg CO2/gallon)': 10.18, 'hydrogen prod. emission factor (kg CO2-eq/kg)': 0.0, 'formic acid prod. emission factor (kg CO2-eq/kg)': 0.0, 'hydrogen purchase cost ($/kg)': 0.31, 'formic acid purchase cost ($/kg)': 1.0, 'formic acid production pathway': 'electro', 'hydr. reaction temperature (K)': 366.15, 'hydr. reaction pressure (bar)': 105.0, 'hydr. reaction yield': 1.0, 'hydr. reactor volume (m^3)': 1.0, 'number of hydr. reactors': 1, 'hydr. catalyst amount (kg)': 53.0, 'hydr. catalyst cost ($/kg)': 5450.0, 'hydr. catalyst lifetime (yr)': 1.0, 'hydr. reactor energy (unit TBD)': 0.0, 'hydr. separator energy (

### USER INPUT

In [3]:
# specify input parameter file
df_input_params_all = pd.read_excel('inputs/input params_baseline.xlsx')

# specify output folder
output_folder = 'outputs/outputs ' + date.today().strftime("%Y-%m-%d") + ' baseline'

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

### RUN SYSTEMS ANALYSIS

In [ ]:
# create output folder if not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [ ]:
# fill NaN values in input parameter file
df_input_params_all.ffill(inplace = True)

df_input_params_all

In [ ]:
# create output dataframe for all runs
df_output_all = pd.DataFrame()

# run systems analysis for each set of input parameters
for i in df_input_params_all.index:
    
    # create dictionary of input parameters for each run
    df_input_params = df_input_params_all.iloc[i]
    dict_input_params = df_input_params.to_dict()

    # run systems analysis
    df_output, _, _, _ = s2a_sys.calcs(
        dict_input_params, 
        save_csv = True,
        output_folder = output_folder
        )

    # add run number column and rearrange columns
    cols = df_output.columns.tolist()
    df_output['run #'] = df_input_params['run #']
    df_output = df_output[['run #'] + cols]

    # add output dataframe to dataframe for all runs
    df_output_all = pd.concat([df_output_all, df_output])

In [ ]:
# save output dataframe for all runs as csv
df_output_all.to_csv(
    os.path.join(output_folder, 'output_all runs.csv'),         
    index = False
    )

df_output_all

### PROCESS OUTPUT

In [ ]:
# drop scenario and run number rows
df_output_all = df_output_all.loc[
    ~df_output_all['variable name'].isin(['scenario', 'run #'])
    ].reset_index(drop = True)

In [ ]:
# filter for $/kg H2 cost results
df_usd_per_kg = df_output_all.loc[
    df_output_all['unit'] == '$/kg H2'
    ].reset_index(drop = True)

# filter for kg CO2/kg H2 emissions results
df_kgCO2_per_kg = df_output_all.loc[
    df_output_all['unit'] == 'kg CO2/kg H2'
    ].reset_index(drop = True)

#### OUTPUT: all input parameters

In [ ]:
# write input parameters as csv to output folder
df_input_params_all.to_csv(
    os.path.join(output_folder, 'input params.csv'),         
    index = False
    )

#### OUTPUT: varied input parameters
including base and user-defined values 

In [ ]:
# find column names of varied input parameters
varied_cols = []

for col in df_input_params_all.columns[2:]:
    if any(df_input_params_all[col] != df_input_params_all[col][0]):
        varied_cols.append(col)

# extract varied input parameters
df_input_params_varied = df_input_params_all[
    ['scenario', 'run #'] + varied_cols
    ]

# save dataframe as csv
df_input_params_varied.to_csv(
    os.path.join(output_folder, 'input params_varied.csv'),         
    index = False
    )

df_input_params_varied

#### OUTPUT: refueling station costs for process modeling team

In [ ]:
# set filters for refueling station costs
# including catalyst amount and reactor volume
pathway = ['LOHC - formic acid', 'all']
location = ['refueling station', 'all']
units = ['$/yr', '$/kg H2', '$/station', 'kg', 'm^3']
# equipment = [
#     'compressor',
#     'refrigerator',
#     'cryogenic pump',
#     'reactor pump',
#     'reactor',
#     'catalyst',
#     'PSA refrigerator',
#     'PSA'
#     ]

# filter results
df_stn = df_output_all.loc[
    df_output_all['pathway'].isin(pathway) & \
    df_output_all['location'].isin(location) & \
    df_output_all['unit'].isin(units)
    ].reset_index(drop = True)

# filter out hydrogenation inputs
df_stn = df_stn.loc[
    ~df_stn['variable name'].str.startswith('hydr.')
    ]

# save dataframe as csv
df_stn.to_csv(
    os.path.join(
        output_folder, 
        'output_all runs_refueling station costs.csv'
        ),         
    index = False
    )

df_stn

In [ ]:
# create column of output name
df_stn['name'] = \
    df_stn['equipment'] + ' ' + \
    df_stn['variable name'] + ' (' + \
    df_stn['unit'] + ')'

# drop unused columns
df_stn = df_stn[[
    'run #', 
    'name',
    'value',
    ]]

# pivot dataframe
df_stn_pivot = df_stn.pivot(
    columns = 'name', 
    index = 'run #', 
    values = 'value'
    ).reset_index()

# remove name of columns
df_stn_pivot.columns.name = None

# merge in varied input parameters
df_stn_pivot = df_input_params_varied.merge(
    right = df_stn_pivot, 
    on='run #', 
    how = 'right'
    )

# save dataframe as csv
df_stn_pivot.to_csv(
    os.path.join(
        output_folder, 
        'output_all runs_refueling station costs_pivot.csv'
        ),         
    index = False
    )

df_stn_pivot

#### OUTPUT: total costs ($/kg H2) and emissions (kg CO2/kg H2) by pathway

In [ ]:
# calculate total $/kg H2 cost by pathway
df_usd_per_kg_path = df_usd_per_kg.groupby(
    by = ['run #', 'pathway'])['value'].sum().reset_index()

# update column name
df_usd_per_kg_path.rename(
    columns = {'value' : 'total levelized cost of hydrogen ($/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_usd_per_kg_path = df_input_params_all.merge(
    right = df_usd_per_kg_path, 
    on = 'run #'
    )

# save dataframe as csv
df_usd_per_kg_path.to_csv(
    os.path.join(output_folder, 'output_cost by pathway.csv'),         
    index = False
    )

df_usd_per_kg_path

In [ ]:
# calculate total kg CO2/kg H2 emissions by pathway
df_kgCO2_per_kg_path = df_kgCO2_per_kg.groupby(
    by = ['run #', 'pathway'])['value'].sum().reset_index()

# update column name
df_kgCO2_per_kg_path.rename(
    columns = {'value' : 'total emissions (kg CO2-eq/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_kgCO2_per_kg_path = df_input_params_all.merge(
    right = df_kgCO2_per_kg_path, 
    on = 'run #'
    )

# save dataframe as csv
df_kgCO2_per_kg_path.to_csv(
    os.path.join(output_folder, 'output_emissions by pathway.csv'),         
    index = False
    )

df_kgCO2_per_kg_path

#### OUTPUT: total costs ($/kg H2) and emissions (kg CO2/kg H2) by pathway and process

In [ ]:
# calculate total $/kg H2 cost by pathway and process
df_usd_per_kg_proc = df_usd_per_kg.groupby(
    by = ['run #', 'pathway', 'process'])['value'].sum().reset_index()

# update column name
df_usd_per_kg_proc.rename(
    columns = {'value' : 'total levelized cost of hydrogen ($/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_usd_per_kg_proc = df_input_params_all.merge(
    right = df_usd_per_kg_proc, 
    on = 'run #'
    )

# save dataframe as csv
df_usd_per_kg_proc.to_csv(
    os.path.join(output_folder, 'output_cost by process.csv'),         
    index = False
    )

df_usd_per_kg_proc

In [ ]:
# calculate total kg CO2/kg H2 emissions by pathway and process
df_kgCO2_per_kg_proc = df_kgCO2_per_kg.groupby(
    by = ['run #', 'pathway', 'process'])['value'].sum().reset_index()

# update column name
df_kgCO2_per_kg_proc.rename(
    columns = {'value' : 'total emissions (kg CO2-eq/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_kgCO2_per_kg_proc = df_input_params_all.merge(
    right = df_kgCO2_per_kg_proc, 
    on = 'run #'
    )

# save dataframe as csv
df_kgCO2_per_kg_proc.to_csv(
    os.path.join(output_folder, 'output_emissions by process.csv'),         
    index = False
    )

df_kgCO2_per_kg_proc

#### OUTPUT: total costs ($/kg H2) and emissions (kg CO2/kg H2) by pathway and location

In [ ]:
# calculate total $/kg H2 cost by pathway and location
df_usd_per_kg_loc = df_usd_per_kg.groupby(
    by = ['run #', 'pathway', 'location'])['value'].sum().reset_index()

# update column name
df_usd_per_kg_loc.rename(
    columns = {'value' : 'total levelized cost of hydrogen ($/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_usd_per_kg_loc = df_input_params_all.merge(
    right = df_usd_per_kg_loc, 
    on = 'run #'
    )

# save dataframe as csv
df_usd_per_kg_loc.to_csv(
    os.path.join(output_folder, 'output_cost by location.csv'),         
    index = False
    )

df_usd_per_kg_loc

In [ ]:
# calculate total kg CO2/kg H2 emissions by pathway and location
df_kgCO2_per_kg_loc = df_kgCO2_per_kg.groupby(
    by = ['run #', 'pathway', 'location'])['value'].sum().reset_index()

# update column name
df_kgCO2_per_kg_loc.rename(
    columns = {'value' : 'total emissions (kg CO2-eq/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_kgCO2_per_kg_loc = df_input_params_all.merge(
    right = df_kgCO2_per_kg_loc, 
    on = 'run #'
    )

# save dataframe as csv
df_kgCO2_per_kg_loc.to_csv(
    os.path.join(output_folder, 'output_emissions by location.csv'),         
    index = False
    )

df_kgCO2_per_kg_loc

#### OUTPUT: total costs ($/kg H2) and emissions (kg CO2/kg H2) by pathway and function

In [ ]:
# calculate total $/kg H2 cost by pathway and function
df_usd_per_kg_func = df_usd_per_kg.groupby(
    by = ['run #', 'pathway', 'function'])['value'].sum().reset_index()

# update column name
df_usd_per_kg_func.rename(
    columns = {'value' : 'total levelized cost of hydrogen ($/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_usd_per_kg_func = df_input_params_all.merge(
    right = df_usd_per_kg_func, 
    on = 'run #'
    )

# save dataframe as csv
df_usd_per_kg_func.to_csv(
    os.path.join(output_folder, 'output_cost by function.csv'),         
    index = False
    )

df_usd_per_kg_func

In [ ]:
# calculate total kg CO2/kg H2 emissions by pathway and function
df_kgCO2_per_kg_func = df_kgCO2_per_kg.groupby(
    by = ['run #', 'pathway', 'function'])['value'].sum().reset_index()

# update column name
df_kgCO2_per_kg_func.rename(
    columns = {'value' : 'total emissions (kg CO2-eq/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_kgCO2_per_kg_func = df_input_params_all.merge(
    right = df_kgCO2_per_kg_func, 
    on = 'run #'
    )

# save dataframe as csv
df_kgCO2_per_kg_func.to_csv(
    os.path.join(output_folder, 'output_emissions by function.csv'),         
    index = False
    )

df_kgCO2_per_kg_func

#### OUTPUT: total costs ($/kg H2) and emissions (kg CO2/kg H2) by pathway, process, and function

In [ ]:
# calculate total $/kg H2 cost by pathway, process, and function
df_usd_per_kg_proc_func = df_usd_per_kg.groupby(
    by = ['run #', 'pathway', 'process', 'function']
    )['value'].sum().reset_index()

# update column name
df_usd_per_kg_proc_func.rename(
    columns = {'value' : 'total levelized cost of hydrogen ($/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_usd_per_kg_proc_func = df_input_params_all.merge(
    right = df_usd_per_kg_proc_func, 
    on = 'run #'
    )

# save dataframe as csv
df_usd_per_kg_proc_func.to_csv(
    os.path.join(output_folder, 'output_cost by process-function.csv'),         
    index = False
    )

df_usd_per_kg_proc_func

In [ ]:
# calculate total kg CO2/kg H2 emissions by pathway, process, and function
df_kgCO2_per_kg_proc_func = df_kgCO2_per_kg.groupby(
    by = ['run #', 'pathway', 'process', 'function']
    )['value'].sum().reset_index()

# update column name
df_kgCO2_per_kg_proc_func.rename(
    columns = {'value' : 'total emissions (kg CO2-eq/kg H2)'}, 
    inplace = True
    )
        
# merge with input parameter dataframe
df_kgCO2_per_kg_proc_func = df_input_params_all.merge(
    right = df_kgCO2_per_kg_proc_func, 
    on = 'run #'
    )

# save dataframe as csv
df_kgCO2_per_kg_proc_func.to_csv(
    os.path.join(output_folder, 'output_emissions by process-function.csv'),         
    index = False
    )

df_kgCO2_per_kg_proc_func